In [1]:
import pandas as pd
import numpy as np
from cv_py import CrossValidation
from dataloading import DataLoader
from eda_py import EDA
from visualize_py import Visualize
import seaborn as sns
import matplotlib.pyplot as plt
from implicit.evaluation import train_test_split
from eals import ElementwiseAlternatingLeastSquares, load_model

In [3]:
%cd C:\Users\781110104\OneDrive - Genpact\Documents\VSCode

C:\Users\781110104\OneDrive - Genpact\Documents\VSCode


In [4]:
dl = DataLoader()
user_item_co_t = dl.import_data('TEREX', 'CO', 'df', clip=99.999)
user_item_po_t = dl.import_data('TEREX', 'PO', 'df', clip=99.999)
user_item_co_a = dl.import_data('AGCO', 'CO', 'df', clip=99.999)
user_item_po_a = dl.import_data('AGCO', 'PO', 'df', clip=99.999)

In [5]:
user_item_csr_t = dl.to_csr(user_item_co_t)
user_item_csr_a = dl.to_csr(user_item_co_a)

In [6]:
cv_t = CrossValidation(user_item_csr_t, 5)
cv_a = CrossValidation(user_item_csr_a, 5)

In [7]:
test_t, train_t = cv_t.split_k_fold()
test_a, train_a = cv_a.split_k_fold()

In [9]:
alpha = (user_item_csr_a.shape[0] * user_item_csr_a.shape[1] - user_item_csr_a.nnz) / sum(user_item_csr_a.data)
alpha

3.184593014418637

In [11]:
import implicit


model_cpu = implicit.als.AlternatingLeastSquares(factors=64, regularization=0.01, alpha=3, iterations=15, use_cg=True, use_native=True, use_gpu=False)
model_gpu = implicit.als.AlternatingLeastSquares(factors=64, regularization=0.01, alpha=3, iterations=15, use_cg=True, use_native=True, use_gpu=True)

ValueError: No CUDA extension has been built, can't train on GPU.

In [10]:
space_iALS_a = {'factors' : [64], 'regularization' : [0.01], 'alpha' : [3], 'iterations' : [15]}
hyper_test_als_a = cv_t.hyperp_tuning(test_a, train_a, space_iALS_a, 'iALS')
hyper_test_als_a

,factors,regularization,alpha,iterations,precision,map,ndcg,auc,mpr
0,64,0.01,3,15,0.430959,0.263799,0.418187,0.503818,0.158524
